In [5]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import joblib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
class DeckDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df_file):
        self.df = joblib.load(df_file)
        self.n_feats = len(self.df.iloc[0])
        self.transform = True

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx]
        sample = torch.torch.FloatTensor(sample.values)
        return sample

In [7]:
class DeckNet(nn.Module):
    def __init__(self, n_feats):
        super(DeckNet, self).__init__()
        self.fc1 = nn.Linear(n_feats, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 64)
        self.fc5 = nn.Linear(64, n_feats)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [8]:
deck_data = DeckDataset('data/common_train.dat')
deck_dataloader = DataLoader(deck_data, batch_size=128, shuffle=True, num_workers=0)
deck_net = DeckNet(deck_data.n_feats)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
deck_net = deck_net.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(deck_net.parameters(), lr=0.001, betas=(0.5, 0.999))

test_data = DeckDataset('data/common_test.dat')
deck_testloader = DataLoader(test_data, batch_size=128, shuffle=False, num_workers=0)

In [9]:
for epoch in range(400):
    optimizer = optim.Adam(deck_net.parameters(), lr=0.01/(epoch+1), betas=(0.5, 0.999))
    running_loss = 0.0
    for i, data in enumerate(deck_dataloader):
        x = data.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = deck_net(x)
        loss = criterion(outputs, x)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    #print('train')
    #print('[%d] loss: %.3f' % (epoch + 1, running_loss))
    running_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(deck_testloader):
            x = data.to(device)

            # forward + backward + optimize
            outputs = deck_net(x)
            loss = criterion(outputs, x)
            out = outputs.cpu().numpy()
            running_loss += loss.item()
    #print('test')
    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss*9))

[1] loss: 9.329
[2] loss: 6.946
[3] loss: 6.301
[4] loss: 5.762
[5] loss: 5.510
[6] loss: 5.366
[7] loss: 5.187
[8] loss: 5.076
[9] loss: 5.013
[10] loss: 4.937
[11] loss: 4.870
[12] loss: 4.837
[13] loss: 4.775
[14] loss: 4.737
[15] loss: 4.706
[16] loss: 4.679
[17] loss: 4.656
[18] loss: 4.620
[19] loss: 4.600
[20] loss: 4.583
[21] loss: 4.560
[22] loss: 4.545
[23] loss: 4.531
[24] loss: 4.513
[25] loss: 4.504
[26] loss: 4.481
[27] loss: 4.476
[28] loss: 4.465
[29] loss: 4.448
[30] loss: 4.442
[31] loss: 4.427
[32] loss: 4.424
[33] loss: 4.411
[34] loss: 4.404
[35] loss: 4.396
[36] loss: 4.392
[37] loss: 4.381
[38] loss: 4.379
[39] loss: 4.371
[40] loss: 4.366
[41] loss: 4.357
[42] loss: 4.352
[43] loss: 4.349
[44] loss: 4.345
[45] loss: 4.337
[46] loss: 4.335
[47] loss: 4.330
[48] loss: 4.325
[49] loss: 4.321
[50] loss: 4.318
[51] loss: 4.313
[52] loss: 4.311
[53] loss: 4.308
[54] loss: 4.304
[55] loss: 4.298
[56] loss: 4.296
[57] loss: 4.292
[58] loss: 4.290
[59] loss: 4.287
[60] l

In [10]:
a = deck_net.fc1.weight.cpu().detach().numpy().T
np.savetxt('data/encoder_embedding.csv', a, delimiter=',')
b = deck_net.fc5.weight.cpu().detach().numpy()
np.savetxt('data/decoder_embedding.csv', b, delimiter=',')

In [15]:
c = joblib.load('data/common_inv.dat')

In [16]:
dot = {}
for i in range(len(a)):
    print(c.get(i, 'Other'))
    dot[i] = np.dot(a, a[i])
    ind = np.argpartition(dot[i], -15)[-15:]
    for j in ind:
        print(j, c.get(j, 'Other'), dot[i][j])
    print()

Other
447 Spectral Sailor 0.36023757
13 Island 0.5931894
85 Plains 0.5488522
17 Swamp 0.5273806
130 Glacial Fortress 0.3897012
18 Thoughtseize 0.38594824
11 Forest 0.46266848
10 Fatal Push 0.4358818
9 Fabled Passage 0.42436674
24 Mystical Dispute 0.58258307
33 Mountain 0.56667477
190 Favored Hoplite 0.38765603
50 Drowned Catacomb 0.3785981
196 Collected Company 0.39580923
0 Other 1.5184684

Abrupt Decay
687 Aphemia, the Cacophony 0.09180714
372 Paradise Druid 0.100290105
652 Uro, Titan of Nature's Wrath 0.09597059
1 Abrupt Decay 0.15563135
13 Island 0.13595802
696 Collective Defiance 0.09182395
9 Fabled Passage 0.115715675
33 Mountain 0.09327433
164 Chained to the Rocks 0.09987608
123 Sylvan Caryatid 0.16186428
24 Mystical Dispute 0.11889346
75 Teferi, Time Raveler 0.10393205
258 Bring to Light 0.14151417
262 Niv-Mizzet Reborn 0.12840869
0 Other 0.16495384

Assassin's Trophy
561 Sylvan Scrying 0.048409827
292 Lovestruck Beast 0.048658267
247 Chemister's Insight 0.04890661
560 Lotus Fie

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.08354431
76 Temple Garden 0.07953626
50 Drowned Catacomb 0.082999125
10 Fatal Push 0.08570728
52 Hinterland Harbor 0.108346626
9 Fabled Passage 0.11681896
5 Breeding Pool 0.09414621
24 Mystical Dispute 0.095897004
0 Other 0.15670273

Woodland Cemetery
4 Botanical Sanctum 0.053065486
102 Gifted Aetherborn 0.054487806
500 Monastery Mentor 0.055597167
62 Negate 0.053547196
16 Overgrown Tomb 0.05376783
302 Zetalpa, Primal Dawn 0.05590604
177 Thraben Inspector 0.061077833
328 Haunted Dead 0.057061046
740 Turntimber Symbiosis 0.056984436
123 Sylvan Caryatid 0.060105365
300 Lotleth Troll 0.07058301
480 Sinister Sabotage 0.05806087
21 Watery Grave 0.05908847
297 Gather the Pack 0.061625358
53 Woodland Cemetery 0.0882833

Narset, Parter of Veils
17 Swamp 0.12282557
133 Supreme Verdict 0.12531583
33 Mountain 0.1270599
10 Fatal Push 0.12946549
116 Sacred Foundry 0.13326113
21 Watery Grave 0.13741612
75 Teferi, Time Raveler 0.13822074
130 Glacial Fortress 0.14600039
18 Thoughtseize 0.14662161
24

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Voracious Hydra 0.085398816
70 Oath of Nissa 0.08630411
660 Wolfwillow Haven 0.08745928
564 Dusk Legion Zealot 0.09494775
106 Sorin, Imperious Bloodlord 0.107559115
740 Turntimber Symbiosis 0.11443614
11 Forest 0.09809408
122 Nissa, Who Shakes the World 0.08709706
168 Vivien, Arkbow Ranger 0.10026717
101 Champion of Dusk 0.104742415
105 Nykthos, Shrine to Nyx 0.14158921

Sorin, Imperious Bloodlord
104 Knight of the Ebon Legion 0.10834674
100 Castle Locthwain 0.109521925
23 Kalitas, Traitor of Ghet 0.12358554
24 Mystical Dispute 0.1253175
262 Niv-Mizzet Reborn 0.12315244
674 Omen of the Sea 0.11521409
645 Drill Bit 0.13368027
17 Swamp 0.22392543
10 Fatal Push 0.14826223
33 Mountain 0.1399393
564 Dusk Legion Zealot 0.4168035
106 Sorin, Imperious Bloodlord 0.45662183
102 Gifted Aetherborn 0.23890598
101 Champion of Dusk 0.44826782
0 Other 0.18023388

Urborg, Tomb of Yawgmoth
347 Aether Gust 0.06336945
7 Dig Through Time 0.06374786
237 Murderous Rider 0.0642826
223 Castle Ardenvale 0.06475

...

0.07027548
101 Champion of Dusk 0.07534379
623 Vizier of Tumbling Sands 0.06525482
399 Arboreal Grazer 0.07461196
712 Extinction Event 0.06491592
10 Fatal Push 0.06568354
17 Swamp 0.07747307
7 Dig Through Time 0.09043249
102 Gifted Aetherborn 0.08335005
564 Dusk Legion Zealot 0.07786085
0 Other 0.09194088

Kari Zev, Skyship Raider
9 Fabled Passage 0.076348834
716 Shark Typhoon 0.07661761
237 Murderous Rider 0.078997634
504 Captain Lannery Storm 0.0803026
18 Thoughtseize 0.082378745
190 Favored Hoplite 0.08158117
164 Chained to the Rocks 0.08272206
10 Fatal Push 0.090744235
13 Island 0.1131119
24 Mystical Dispute 0.1189659
85 Plains 0.113041356
193 Reckless Rage 0.08839118
17 Swamp 0.12409069
33 Mountain 0.14334832
0 Other 0.1817244

Nullhide Ferox
247 Chemister's Insight 0.057600304
21 Watery Grave 0.059884965
196 Collected Company 0.06911804
50 Drowned Catacomb 0.062487505
4 Botanical Sanctum 0.06466043
116 Sacred Foundry 0.066555016
263 Oath of Kaya 0.05911015
575 Nullhide Ferox 0.06

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 0.07205663
392 Steel Overseer 0.0548516
536 Bolas's Citadel 0.055842422
106 Sorin, Imperious Bloodlord 0.072774
101 Champion of Dusk 0.062963106
7 Dig Through Time 0.0628124
311 Zulaport Cutthroat 0.059993055
626 Mayhem Devil 0.06619914
643 Priest of Forgotten Gods 0.05537526
671 Woe Strider 0.079539165
38 Strategic Planning 0.054610938
0 Other 0.07788241

Soul-Guide Lantern
9 Fabled Passage 0.11801216
42 Wild Slash 0.12168235
33 Mountain 0.12361431
189 Dreadhorde Arcanist 0.12527506
672 Soul-Guide Lantern 0.15359113
193 Reckless Rage 0.12426131
24 Mystical Dispute 0.1438146
18 Thoughtseize 0.14580463
17 Swamp 0.17689893
345 Growth Spiral 0.13942856
13 Island 0.12936899
731 Village Rites 0.13758633
85 Plains 0.14069173
95 Duress 0.12874259
0 Other 0.21092026

Shatter the Sky
196 Collected Company 0.07605822
17 Swamp 0.07827276
562 Thespian's Stage 0.07937097
673 Shatter the Sky 0.08201794
560 Lotus Field 0.07980841
50 Drowned Catacomb 0.08274597
33 Mountain 0.082691714
13 Island 0.084

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Turntimber Symbiosis 0.13752422
767 Tangled Florahedron 0.15890463
748 Sea Gate Restoration 0.11793132
0 Other 0.114536166

Valakut Awakening
346 Wilderness Reclamation 0.037398957
129 Clifftop Retreat 0.038826507
136 Gideon of the Trials 0.03754494
517 Fires of Invention 0.037979268
630 Shimmer of Possibility 0.03804375
726 Lofty Denial 0.040273983
37 Steam Vents 0.042706277
706 Indatha Triome 0.041513104
320 Diregraf Ghoul 0.040880233
402 Karn, the Great Creator 0.043518584
259 Dreadbore 0.044825837
247 Chemister's Insight 0.047408897
127 Castle Vantress 0.05567172
768 Valakut Awakening 0.072896734
116 Sacred Foundry 0.045176737

Vastwood Fortification
271 Worldspine Wurm 0.04792476
372 Paradise Druid 0.05242899
757 Pelakka Predation 0.072535604
758 Undercity Informer 0.07721634
756 Balustrade Spy 0.085637204
750 Hagra Mauling 0.06144116
749 Agadeem's Awakening 0.061413184
748 Sea Gate Restoration 0.061987255
743 Bala Ged Recovery 0.076002665
740 Turntimber Symbiosis 0.062470067
123 

In [17]:
dot = {}
for i in range(len(b)):
    print(c.get(i, 'Other'))
    dot[i] = np.dot(b, b[i])
    ind = np.argpartition(dot[i], -15)[-15:]
    for j in ind:
        print(j, c.get(j, 'Other'), dot[i][j])
    print()

Other
444 Firedrinker Satyr 0.102357134
745 Genesis Ultimatum 0.10406083
702 Spark Double 0.10563822
297 Gather the Pack 0.113170214
301 Soulflayer 0.10733904
78 Deafening Clarion 0.11056545
136 Gideon of the Trials 0.10634677
746 Lotus Cobra 0.119335346
698 Elspeth's Nightmare 0.14845075
506 Ghor-Clan Rampager 0.12211389
17 Swamp 0.24059671
458 Thought-Knot Seer 0.13920304
5 Breeding Pool 0.13488427
700 Altered Ego 0.1669521
0 Other 1.9652889

Abrupt Decay
18 Thoughtseize 0.2784992
258 Bring to Light 0.28593782
400 Neoform 0.2983204
262 Niv-Mizzet Reborn 0.30597493
767 Tangled Florahedron 0.3185315
372 Paradise Druid 0.32623696
756 Balustrade Spy 0.30469373
758 Undercity Informer 0.32393086
329 Prized Amalgam 0.3517798
16 Overgrown Tomb 0.4044395
1 Abrupt Decay 0.5834687
411 Creeping Chill 0.41207412
748 Sea Gate Restoration 0.3625545
749 Agadeem's Awakening 0.39544004
123 Sylvan Caryatid 0.6820389

Assassin's Trophy
301 Soulflayer 0.09952852
385 Shambling Vent 0.101674065
298 Grisly 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 Fabled Passage 0.21232367
52 Hinterland Harbor 0.32662386
127 Castle Vantress 0.15688872
346 Wilderness Reclamation 0.18887138
652 Uro, Titan of Nature's Wrath 0.26701975
5 Breeding Pool 0.16460524
728 Necromentia 0.12749171

Woodland Cemetery
10 Fatal Push 0.08685939
101 Champion of Dusk 0.08689303
302 Zetalpa, Primal Dawn 0.087337896
301 Soulflayer 0.089645535
16 Overgrown Tomb 0.2654599
53 Woodland Cemetery 0.1902746
652 Uro, Titan of Nature's Wrath 0.116050385
177 Thraben Inspector 0.14461556
292 Lovestruck Beast 0.09510214
514 Temple of Silence 0.1071454
636 Nightpack Ambusher 0.09046124
71 Reflector Mage 0.101911195
1 Abrupt Decay 0.09848015
3 Blooming Marsh 0.14008778
31 Izzet Charm 0.10174672

Narset, Parter of Veils
44 Anger of the Gods 0.2494356
39 Sulfur Falls 0.26168084
24 Mystical Dispute 0.26554203
715 Raugrin Triome 0.27218807
675 The Birth of Meletis 0.27613252
680 Elspeth Conquers Death 0.29544756
118 Dovin's Veto 0.29568556
75 Teferi, Time Raveler 0.28963843
716 Shar

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




734 Llanowar Visionary 0.14917004
70 Oath of Nissa 0.22409779
13 Island 0.21614628
168 Vivien, Arkbow Ranger 0.24889979
402 Karn, the Great Creator 0.29908365
105 Nykthos, Shrine to Nyx 0.41965434
660 Wolfwillow Haven 0.2603417
11 Forest 0.546127
122 Nissa, Who Shakes the World 0.260163
170 Voracious Hydra 0.2495755

Sorin, Imperious Bloodlord
17 Swamp 0.106198594
132 Irrigated Farmland 0.11232048
139 Burning-Tree Emissary 0.10879488
110 Grasp of Darkness 0.115186594
674 Omen of the Sea 0.15343082
133 Supreme Verdict 0.120080814
675 The Birth of Meletis 0.118598945
28 Arclight Phoenix 0.12691335
129 Clifftop Retreat 0.11595725
116 Sacred Foundry 0.15993343
23 Kalitas, Traitor of Ghet 0.16111444
102 Gifted Aetherborn 0.41561374
564 Dusk Legion Zealot 0.3798625
106 Sorin, Imperious Bloodlord 0.48263705
101 Champion of Dusk 0.4867179

Urborg, Tomb of Yawgmoth
32 Lightning Axe 0.07895683
645 Drill Bit 0.080148876
71 Reflector Mage 0.08214832
100 Castle Locthwain 0.085044965
102 Gifted Aet

...

 Clever Impersonator 0.070301786
595 Gurmag Angler 0.06790546
599 Cavalier of Thorns 0.07294153
281 Spire of Industry 0.09541279
584 Diregraf Colossus 0.06887665

Nebelgast Herald
45 Brazen Borrower 0.1611321
5 Breeding Pool 0.22074302
76 Temple Garden 0.31229872
196 Collected Company 0.29066026
114 Hallowed Fountain 0.2509029
130 Glacial Fortress 0.22748923
204 Selfless Spirit 0.3540243
4 Botanical Sanctum 0.22482537
447 Spectral Sailor 0.3754966
206 Supreme Phantom 0.45746607
201 Rattlechains 0.4346336
200 Mausoleum Wanderer 0.402804
199 Empyrean Eagle 0.3980118
566 Nebelgast Herald 0.44193172
138 Spell Queller 0.4208635

Season of Growth
107 Urborg, Tomb of Yawgmoth 0.068459
377 Inspiring Veteran 0.06900197
299 Llanowar Wastes 0.07395529
663 Dream Trawler 0.07257281
669 Alseid of Life's Bounty 0.074641
123 Sylvan Caryatid 0.078091055
751 Feed the Swarm 0.07631889
241 Hangarback Walker 0.08376137
567 Season of Growth 0.08401493
198 Dromoka's Command 0.073234305
397 Apostle of Purifyi

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




246 Azorius Charm 0.22190505
85 Plains 0.13855778
34 Opt 0.2124289
334 Ghostfire Blade 0.12885159

Klothys, God of Destiny
336 Pia and Kiran Nalaar 0.06797882
14 Jace, Vryn's Prodigy 0.06835985
257 Blood Crypt 0.07216552
758 Undercity Informer 0.07030024
292 Lovestruck Beast 0.07692763
731 Village Rites 0.07849287
222 Brave the Elements 0.07907689
177 Thraben Inspector 0.08207926
258 Bring to Light 0.08465683
713 General Kudro of Drannith 0.08614832
664 Klothys, God of Destiny 0.10302253
268 Game Trail 0.079893835
269 Rootbound Crag 0.102390565
74 Stomping Ground 0.15013997
71 Reflector Mage 0.10004546

Heliod, Sun-Crowned
630 Shimmer of Possibility 0.081099465
454 Ipnu Rivulet 0.08483404
568 The Great Henge 0.09837285
321 Dread Wanderer 0.08968386
736 Transmogrify 0.08398801
380 Fortified Village 0.082715176
590 Hanweir Garrison 0.085223705
725 Eliminate 0.11219007
263 Oath of Kaya 0.08632236
665 Heliod, Sun-Crowned 0.14972842
292 Lovestruck Beast 0.090061
676 Treacherous Blessing 0.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



750 Hagra Mauling 0.15757644

Clearwater Pathway
252 Syncopate 0.061768293
213 Cartouche of Solidarity 0.0625632
480 Sinister Sabotage 0.06362788
55 Harbinger of the Tides 0.063747324
427 Dissenter's Deliverance 0.074216865
575 Nullhide Ferox 0.06615347
388 Niv-Mizzet, Parun 0.06697621
648 Void Snare 0.071636446
683 Erebos's Intervention 0.08943958
2 Assassin's Trophy 0.064920925
765 Clearwater Pathway 0.09291469
361 Supreme Will 0.07210704
50 Drowned Catacomb 0.07796843
320 Diregraf Ghoul 0.065811865
308 Cartel Aristocrat 0.072262324

Archon of Emeria
766 Archon of Emeria 0.052122746
749 Agadeem's Awakening 0.053729367
118 Dovin's Veto 0.058098823
143 Natural State 0.05870033
52 Hinterland Harbor 0.058757134
756 Balustrade Spy 0.06149807
400 Neoform 0.06643236
748 Sea Gate Restoration 0.067720406
18 Thoughtseize 0.07667099
750 Hagra Mauling 0.08806451
610 Eldritch Evolution 0.082457155
372 Paradise Druid 0.11074251
767 Tangled Florahedron 0.08833524
411 Creeping Chill 0.08261641
729 S

In [144]:
ind = np.argpartition(abrupt_decay, -15)[-15:]

In [146]:
for i in ind:
    print(i, c[i], abrupt_decay[i])

19 Thoughtseize 0.2523992
266 Dreadbore 0.25967532
3 Blooming Marsh 0.26451477
779 Tangled Florahedron 0.26974845
188 Traverse the Ulvenwald 0.2733314
662 Uro, Titan of Nature's Wrath 0.34498245
265 Bring to Light 0.4490811
269 Niv-Mizzet Reborn 0.41657078
129 Sylvan Caryatid 0.76756626
761 Agadeem's Awakening 0.28443494
17 Overgrown Tomb 0.38555396
755 Bala Ged Recovery 0.2880368
380 Paradise Druid 0.29129413
1 Abrupt Decay 0.759647
419 Creeping Chill 0.2755574
